# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-09 23:10:25] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=36145, mem_fraction_static=0.836, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=680662221, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gp

[2025-06-09 23:10:25] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-09 23:10:35] Attention backend not set. Use flashinfer backend by default.
[2025-06-09 23:10:35] Init torch distributed begin.


[2025-06-09 23:10:36] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-09 23:10:41] Init torch distributed begin.
[2025-06-09 23:10:41] Init torch distributed ends. mem usage=0.00 GB
[2025-06-09 23:10:41] Load weight begin. avail mem=55.16 GB
[2025-06-09 23:10:41] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.41s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.41s/it]

[2025-06-09 23:10:42] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-06-09 23:10:42] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-09 23:10:42] Memory pool end. avail mem=53.92 GB


[2025-06-09 23:10:43] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.14 GB
[2025-06-09 23:10:43] INFO:     Started server process [369949]
[2025-06-09 23:10:43] INFO:     Waiting for application startup.
[2025-06-09 23:10:43] INFO:     Application startup complete.
[2025-06-09 23:10:43] INFO:     Uvicorn running on http://127.0.0.1:36145 (Press CTRL+C to quit)


[2025-06-09 23:10:44] INFO:     127.0.0.1:46024 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-09 23:10:44] INFO:     127.0.0.1:46036 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-09 23:10:44] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:10:46] INFO:     127.0.0.1:46048 - "POST /generate HTTP/1.1" 200 OK
[2025-06-09 23:10:46] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-09 23:10:49] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:10:49] INFO:     127.0.0.1:46062 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-09 23:10:49] Child process unexpectedly failed with exitcode=9. pid=370229
[2025-06-09 23:10:49] Child process unexpectedly failed with exitcode=9. pid=370095


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-09 23:11:02] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=33951, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=618864138, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_

[2025-06-09 23:11:02] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-09 23:11:13] Attention backend not set. Use flashinfer backend by default.
[2025-06-09 23:11:13] Init torch distributed begin.


[2025-06-09 23:11:14] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.29s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.29s/it]

[2025-06-09 23:11:20] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-06-09 23:11:20] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-09 23:11:20] Memory pool end. avail mem=53.92 GB


[2025-06-09 23:11:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB
[2025-06-09 23:11:21] INFO:     Started server process [370966]
[2025-06-09 23:11:21] INFO:     Waiting for application startup.
[2025-06-09 23:11:21] INFO:     Application startup complete.
[2025-06-09 23:11:21] INFO:     Uvicorn running on http://127.0.0.1:33951 (Press CTRL+C to quit)


[2025-06-09 23:11:21] INFO:     127.0.0.1:58090 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-09 23:11:22] INFO:     127.0.0.1:58098 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-09 23:11:22] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:11:24] INFO:     127.0.0.1:58106 - "POST /generate HTTP/1.1" 200 OK
[2025-06-09 23:11:24] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-09 23:11:26] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:11:27] INFO:     127.0.0.1:58122 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-09 23:11:27] Child process unexpectedly failed with exitcode=9. pid=371246


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-09 23:11:38] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=37915, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=507337716, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None

[2025-06-09 23:11:50] Casting torch.bfloat16 to torch.float16.


[2025-06-09 23:11:50] Casting torch.bfloat16 to torch.float16.
[2025-06-09 23:11:50] Attention backend not set. Use flashinfer backend by default.
[2025-06-09 23:11:50] Init torch distributed begin.


[2025-06-09 23:11:50] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-09 23:12:08] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-06-09 23:12:08] INFO:     Started server process [371989]
[2025-06-09 23:12:08] INFO:     Waiting for application startup.
[2025-06-09 23:12:08] INFO:     Application startup complete.
[2025-06-09 23:12:08] INFO:     Uvicorn running on http://127.0.0.1:37915 (Press CTRL+C to quit)


[2025-06-09 23:12:09] INFO:     127.0.0.1:45292 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-09 23:12:09] INFO:     127.0.0.1:45308 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-09 23:12:09] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:12:12] INFO:     127.0.0.1:45320 - "POST /generate HTTP/1.1" 200 OK
[2025-06-09 23:12:12] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-09 23:12:14] Detected chat template content format: string
[2025-06-09 23:12:14] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:12:14] INFO:     127.0.0.1:50838 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-09 23:12:14] Child process unexpectedly failed with exitcode=9. pid=372270


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-09 23:12:25] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32412, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=366807410, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gp

[2025-06-09 23:12:35] Casting torch.bfloat16 to torch.float16.


[2025-06-09 23:12:36] Casting torch.bfloat16 to torch.float16.
[2025-06-09 23:12:36] Attention backend not set. Use flashinfer backend by default.
[2025-06-09 23:12:36] Init torch distributed begin.


[2025-06-09 23:12:36] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-09 23:12:54] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.94 GB


[2025-06-09 23:12:54] INFO:     Started server process [373041]
[2025-06-09 23:12:54] INFO:     Waiting for application startup.
[2025-06-09 23:12:54] INFO:     Application startup complete.
[2025-06-09 23:12:54] INFO:     Uvicorn running on http://127.0.0.1:32412 (Press CTRL+C to quit)


[2025-06-09 23:12:55] INFO:     127.0.0.1:45264 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-09 23:12:55] INFO:     127.0.0.1:45274 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-09 23:12:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:12:59] INFO:     127.0.0.1:45276 - "POST /generate HTTP/1.1" 200 OK
[2025-06-09 23:12:59] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-09 23:13:00] Detected chat template content format: string
[2025-06-09 23:13:00] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:13:01] INFO:     127.0.0.1:52328 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-09 23:13:01] Child process unexpectedly failed with exitcode=9. pid=373322


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-09 23:13:11] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=36225, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=600126347, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info'

[2025-06-09 23:13:24] Attention backend not set. Use flashinfer backend by default.
[2025-06-09 23:13:24] Init torch distributed begin.


[2025-06-09 23:13:25] Init torch distributed ends. mem usage=0.00 GB


[2025-06-09 23:13:26] Load weight begin. avail mem=78.50 GB


...

[2025-06-09 23:13:33] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-09 23:13:33] INFO:     Started server process [374076]
[2025-06-09 23:13:33] INFO:     Waiting for application startup.
[2025-06-09 23:13:33] INFO:     Application startup complete.
[2025-06-09 23:13:33] INFO:     Uvicorn running on http://0.0.0.0:36225 (Press CTRL+C to quit)


[2025-06-09 23:13:34] INFO:     127.0.0.1:59196 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-09 23:13:34] INFO:     127.0.0.1:59206 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-09 23:13:34] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:13:36] INFO:     127.0.0.1:59218 - "POST /generate HTTP/1.1" 200 OK
[2025-06-09 23:13:36] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-09 23:13:39] Detected chat template content format: string
[2025-06-09 23:13:39] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-09 23:13:39] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 11.38, #queue-req: 0


[2025-06-09 23:13:40] Decode batch. #running-req: 1, #token: 165, token usage: 0.01, accept len: 1.82, cuda graph: False, gen throughput (token/s): 125.45, #queue-req: 0


[2025-06-09 23:13:40] Decode batch. #running-req: 1, #token: 239, token usage: 0.01, accept len: 1.85, cuda graph: False, gen throughput (token/s): 124.73, #queue-req: 0


[2025-06-09 23:13:41] Decode batch. #running-req: 1, #token: 309, token usage: 0.02, accept len: 1.75, cuda graph: False, gen throughput (token/s): 116.09, #queue-req: 0


...

[2025-06-09 23:13:42] Decode batch. #running-req: 1, #token: 448, token usage: 0.02, accept len: 1.75, cuda graph: False, gen throughput (token/s): 116.70, #queue-req: 0


[2025-06-09 23:13:43] Decode batch. #running-req: 1, #token: 516, token usage: 0.03, accept len: 1.70, cuda graph: False, gen throughput (token/s): 111.48, #queue-req: 0


[2025-06-09 23:13:43] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, accept len: 1.62, cuda graph: False, gen throughput (token/s): 108.23, #queue-req: 0


[2025-06-09 23:13:44] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, accept len: 1.85, cuda graph: False, gen throughput (token/s): 125.77, #queue-req: 0


[2025-06-09 23:13:45] INFO:     127.0.0.1:59230 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-09 23:13:45] Child process unexpectedly failed with exitcode=9. pid=374357


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).